
# Wrapping Arbitrary Classifier or Regressor

This example requires full licence, and the program will break if you use the trial licence.


Installation



In [ ]:
# To install the required package, use the following command:
# pip install modeva

Authentication



In [ ]:
# To get authentication, use the following command: (To get full access please replace the token to your own token)
# from modeva.utils.authenticate import authenticate
# authenticate(token='eaaa4301-b140-484c-8e93-f9f633c8bacb')

Import required modules



In [ ]:
import numpy as np
import pandas as pd
from modeva import DataSet
from modeva import TestSuite
from modeva.models.wrappers.api import modeva_arbitrary_classifier, modeva_arbitrary_regressor
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

## Scripts to build a model



In [ ]:
data = load_breast_cancer()
X, y = data.data, data.target  # Use California housing dataset
train_idx, test_idx = train_test_split(np.arange(data.data.shape[0]),
                                       test_size=0.2, random_state=42)

estimator = LGBMClassifier(verbose=-1)
estimator.fit(X[train_idx], y[train_idx])

## Wrap the data into Modeva



In [ ]:
ds = DataSet()
ds.load_dataframe(pd.concat([pd.DataFrame(data.data, columns=data.feature_names),
                  pd.DataFrame(data.target, columns=[data.target_names[1]])], axis=1))
ds.set_train_idx(train_idx)
ds.set_test_idx(test_idx)

## Wrap the model into Modeva



In [ ]:
def predict_func(X):
    return estimator.predict(X)

def predict_proba_func(X):
    # predict_proba is only needed for classification tasks
    # X should be numpy array, and output should be of shape (X.shape[0], 2)
    return estimator.predict_proba(X)

model = modeva_arbitrary_classifier(name="LGBM-arbitrary",
                                    predict_function=predict_func,
                                    predict_proba_function=predict_proba_func)

## Create test suite for diagnostics



In [ ]:
ts = TestSuite(ds, model)

Permutation feature importance



In [ ]:
result = ts.explain_pfi()
result.plot()

LIME for local explanation



In [ ]:
result = ts.explain_lime(sample_index=0)
result.plot()